In [ ]:
#Step 1: Prepare the DataFrame
#First, let's create a sample DataFrame with columns "music_id", "music_name", and "music_description".
import pandas as pd

# data = {
#     "music_id": [1, 2, 3, 4, 5],
#     "music_name": ["Song 1", "Song 2", "Song 3", "Song 4", "Song 5"],
#     "music_description": [
#         "A beautiful ballad with heartfelt lyrics.",
#         "An upbeat pop song that will make you dance.",
#         "A classic rock anthem with powerful guitar riffs.",
#         "A soulful R&B track with smooth vocals.",
#         "An electronic dance track with pulsing beats."
#     ]
# }

# df = pd.DataFrame(data)
# print(df)

In [ ]:
data = pd.read_csv("/content/annotated_classical_music_dataset.csv")
data.rename(columns={'name_movement': 'music_name', 'keywords': 'music_description','id': 'music_id'}, inplace=True)
df = pd.DataFrame(data.head(10))
print(df)

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"hefugu","key":"324ddef280041fb43790eb61df650409"}')

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d imsparsh/musicnet-dataset
!unzip musicnet-dataset.zip

In [ ]:
!pip install farm-haystack[faiss]

In [ ]:
!pip install faiss-cpu

In [ ]:
# Initialize the FAISSDocumentStore
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",  # Change this to your configuration needs
    sql_url="sqlite:///faiss_inde.db",  # Specify your SQL URL here
    return_embedding=True
)

# If needed, delete existing documents and embeddings
document_store.delete_all_documents()

In [ ]:
#Step 3: Convert DataFrame to Documents
#We need to convert the DataFrame rows into Haystack Document objects.
from haystack import Document

documents = []
for _, row in df.iterrows():
    document = Document(
        content=row["music_description"],
        meta={"name": row["music_name"], "id": row["music_id"]}
    )
    documents.append(document)

In [ ]:
!pip install farm-haystack[inference]

In [ ]:
#Step 4: Initialize the Retriever
#We'll initialize the DensePassageRetriever to encode and retrieve the most relevant music documents based on a given query.
from haystack.nodes import DensePassageRetriever
import sentence_transformers

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

In [ ]:
#Step 5: Write Documents to DocumentStore
#We'll delete any existing documents in the DocumentStore and write the new documents. Then, we'll update the embeddings using the retriever.
document_store.delete_documents()
document_store.write_documents(documents)
document_store.update_embeddings(retriever)

In [ ]:
#Step 6: Retrieve Relevant Documents
#Now, we can use the retriever to find the most relevant music documents based on a given query.
query = "uplifting electronic music"

def retrieve_music_document(query):
    # Your RAG retrieval code here
    retrieved_docs = retriever.retrieve(query)
    return retrieved_docs[0]  # Return the most relevant document
retrieved_doc = retrieve_music_document(query)
print(retrieved_doc)

## Music Generation

In [ ]:
# !pip install --upgrade --quiet pip
# !pip install --upgrade --quiet transformers datasets[audio]

In [ ]:
import os
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from datasets import load_dataset

# Load the MusicGen model and processor
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")


In [ ]:
# import torch
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model.to(device);
# unconditional_inputs = model.get_unconditional_inputs(num_samples=1)
# audio_values = model.generate(**unconditional_inputs, do_sample=True, max_new_tokens=256)

In [ ]:
# Specify the folder containing the music files
music_folder = "/content/musicnet/musicnet/train_data"
query = "uplifting electronic music"
retrieved_doc = retrieve_music_document(query)

In [ ]:
!pip install librosa

In [ ]:
import librosa
# Get the music ID from the retrieved document
music_id = retrieved_doc.meta["id"]

# Construct the path to the music file
music_file = os.path.join(music_folder, f"{music_id}.wav")

audio, sampling_rate = librosa.load(music_file, sr=None, duration=30)  # sr=None to preserve the original sampling rate


In [ ]:
import torch
# Load the music file using the datasets library
# dataset = load_dataset("audio", data_files={"audio": music_file})
# sample = dataset["audio"][0]

music_tensor = torch.tensor(audio)
stride = len(music_tensor) // 96000
indices = torch.arange(0, len(music_tensor), stride)
music_tensor = music_tensor[indices]

if len(music_tensor) > 96000:
    music_tensor = music_tensor[:96000]

# Prepare the inputs for music generation
inputs = processor(
    audio=music_tensor,
    sampling_rate=32000,
    text=[query],
    padding=True,
    return_tensors="pt",
)

In [ ]:
print(music_file)

In [ ]:
# Generate the music
audio_values = model.generate(**inputs, max_new_tokens=32)

In [ ]:
#Play or save the generated audio:
#To play the audio in a notebook:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)


In [ ]:
#To save the audio as a WAV file
import scipy

scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].cpu().numpy())